In [1]:
import numpy as np
import tensorflow as tf


import matplotlib.pyplot as plt
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2
import sgld_tf

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
cnn = sgld_tf.CNN(train_data, train_labels,
                  eval_data, eval_labels,
                  bs=512, model_dir="/tmp/sgld11",
                 optimize_method="ksgld",
                 learning_rate=0.001)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/sgld11', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f733c0768d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
train_spec = tf.estimator.TrainSpec(input_fn=cnn.train_input_fn, max_steps=12000)
eval_spec = tf.estimator.EvalSpec(input_fn=cnn.eval_input_fn, throttle_secs=10)

In [6]:
tf.estimator.train_and_evaluate(cnn.estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/sgld11/model.ckpt-11001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 11002 into /tmp/sgld11/model.ckpt.
INFO:tensorflow:loss = 0.181927, step = 11001
INFO:tensorflow:global_step/sec: 92.5828
INFO:tensorflow:loss = 0.182943, step = 11101 (1.081 sec)
INFO:tensorflow:global_step/sec: 98.0088
INFO:tensorflow:loss = 0.219384, step = 11201 (1.020 sec)
INFO:tensorflow:global_step/sec: 98.102
INFO:tensorflow:loss = 0.227601, step = 11301 (1.020 sec)
INFO:tensorflow:global_step/sec: 97.8119
INFO:tensorflow:loss = 

In [ ]:
cnn.estimator.train(
    input_fn=cnn.train_input_fn,
    steps=20000);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/sgld11/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 12001 into /tmp/sgld11/model.ckpt.
INFO:tensorflow:loss = 0.214874, step = 12000
INFO:tensorflow:global_step/sec: 93.6591
INFO:tensorflow:loss = 0.0968834, step = 12100 (1.069 sec)
INFO:tensorflow:global_step/sec: 98.6527
INFO:tensorflow:loss = 0.155701, step = 12200 (1.014 sec)
INFO:tensorflow:global_step/sec: 98.8264
INFO:tensorflow:loss = 0.178545, step = 12300 (1.012 sec)
INFO:tensorflow:global_step/sec: 98.5435
INFO:tensorflow:loss = 0.246579, step = 12400 (1.015 sec)
INFO:tensorflow:global_step/sec: 98.8324
INFO:tensorflow:loss = 0.162642, step = 12500 (1.012 sec)
INFO:tensorflow:global_step/sec: 99.1752
INFO:tensorflow:loss = 0.149814, st

In [ ]:
eval_results = cnn.estimator.evaluate(input_fn=cnn.eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-02:19:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/sgld11/model.ckpt-32000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
cnn_kfac = sgld_tf.CNN(train_data, train_labels, eval_data, eval_labels, bs=100, model_dir="/tmp/refactor2_kfac", optimize_method="kfac")

In [ ]:
cnn_kfac.estimator.train(input_fn=cnn_kfac.train_input_fn,steps=20000);

In [ ]:
eval_results = cnn_kfac.estimator.evaluate(input_fn=cnn_kfac.eval_input_fn)
print(eval_results)

In [ ]:
.1/2/2/2/2